**PROYECTO OPCIONAL DS4A**

Integrantes grupo 923:

- Diana Naranjo
- Daniel Osorio

# Librerias

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime
#import calmap

# Descripción del problema: 

La inseguridad en Colombia es un tema de preocupación y revisión, tanto para las entidades gubernamentales, las entidades privadas y la población en general. En las entidades gubernamentales puede incurrir en inconformidad con gobierno actual, ya que las personas pueden sentirse vulnerables. Por el lado de las entidades privadas, tales como aseguradoras, podrían sacar ventajas de información delictiva en el país y por otro lado, ayudaría a que las personas tomen ciertas precauciones. Se plantea realizar un análisis para identificar factores delictivos a nivel nacional, departamental y municipal que ayuden a tomar decisiones, desde los diferentes frentes, que mitiguen la inseguridad. Este caso se abordará específicamente sobre hurtos de automotores, ya que el contexto delictivo es muy amplio.

Bajo este contexto, se plantean preguntas como:
• ¿Cuáles son las ciudades con mayor tasa de hurtos?
• ¿Qué días de la semana y en qué hora suelen presentarse los hurtos?
• ¿Cuál es el tipo de arma más usada por los delincuentes?
• ¿Existe alguna correlación entre los hurtos y factores socioeconómicos, como el desempleo, índices de pobreza, nivel educativo?

# Objetivos

Aplicar conocimientos adquiridos durante el curso para determinar comportamientos y posibles factores que influyen en el hurto de automotores, realizando:

- Integración, limpieza y depuración de bases de datos
- Análisis de información y visualización
- Conclusiones y propuestas de futuros proyectos relacionados.

Restricciones: En la actualidad no se cuenta con un censo municipal de automotores, por lo tanto, las tasas de hurto se deben calcular sobre población proyectada del DANE, y asumir que todos los municipios tienen la misma adquirencia de vehículos.

# Solución

Para dar respuesta a este planteamiento de problema, se siguieron los siguientes pasos

Investigación sobre hurtos de automotores, estadísticas historicas, noticias, entre otras, que nos direccionaran hacia las bases de datos actualizadas, planteamientos de hipotesis e ideas sobre posibles variables que expliquen el hurto.

Selección y descargue de diferentes bases de datos

Base de datos Principal:

Hurtos de Automoviles: https://www.datos.gov.co/Seguridad-y-Defensa/Reporte-Hurto-por-Modalidades-Polic-a-Nacional/9vha-vh9n

Bases Secundarias:

Población DANE: https://www.dane.gov.co/index.php/estadisticas-por-tema/demografia-y-poblacion/proyecciones-de-poblacion

Desempleo: http://www.dane.gov.co/index.php/estadisticas-por-tema/mercado-laboral/mercado-laboral-por-departamentos/mercado-laboral-por-departamento-historicos

Indices de Pobreza: https://www.dane.gov.co/index.php/estadisticas-por-tema/pobreza-y-condiciones-de-vida/pobreza-y-desigualdad/pobreza-monetaria-y-multidimensional-en-colombia-2017#pobreza-monetaria-por-departamentos-2017

DIVIPOLA: https://geoportal.dane.gov.co/servicios/descarga-y-metadatos/datos-geoestadisticos/?cod=112

# Cargue de datos y limpieza

## DIVIPOLA

In [2]:
divipola = pd.read_excel('./DIVIPOLA_Municipios.xlsx', header=10)
divipola = divipola[~divipola['Nombre'].isnull()].rename(
    columns={'Código ':'COD_DPTO',
             'Nombre':'DEPARTAMENTO',
             'Código .1':'COD_MUNIC',
             'Nombre.1':'MUNICIPIO'}).drop(columns='Unnamed: 4')
divipola['COD_MUNIC']=(
    divipola['COD_MUNIC'].astype(int).astype(str)
    .str.split(' ',expand=True)[0]
    .astype(str).str.pad(5, fillchar='0')
)

divipola['DEPARTAMENTO'] = divipola['DEPARTAMENTO'].str.replace(',','')

In [3]:
divipola_dpto = divipola[['COD_DPTO', 'DEPARTAMENTO']].drop_duplicates(['COD_DPTO', 'DEPARTAMENTO'])

## BD_Hurtos

In [89]:
hurtos = pd.read_csv('./Reporte_Hurto_por_Modalidades_Polic_a_Nacional.csv')

In [90]:
hurtos['FECHA HECHO']= pd.to_datetime(
    arg=(
        hurtos['FECHA HECHO'].str.split(' ',expand=True)[0].astype(str)#rename(columns={0:'fecha',1:'hora'})
        .str.pad(6, fillchar='0')
    ),
    format='%d/%m/%Y'# %H:%M:%S.%f
)

In [91]:
hurtos['COD_MUNIC']=hurtos['CODIGO DANE'].astype(str).str.split(' ',expand=True)[0].astype(str).str.pad(8, fillchar='0').str[:5]

In [93]:
hurtos['AÑO']=hurtos["FECHA HECHO"].dt.strftime('%Y')
hurtos['PERIODO']=hurtos["FECHA HECHO"].dt.strftime('%Y-%m')
hurtos['DIA']=hurtos["FECHA HECHO"].dt.strftime('%A')

hurtos.drop(columns=['DEPARTAMENTO','MUNICIPIO','GENERO','GRUPO ETARIO'], inplace=True)

In [94]:
hurtos['ARMAS MEDIOS']=(
    np.where(
        (hurtos['ARMAS MEDIOS']=='NO REPORTA') | 
        (hurtos['ARMAS MEDIOS']=='-'), 
        "NO REPORTADO", 
        hurtos['ARMAS MEDIOS'])
)

hurtos['ARMAS MEDIOS']=(
    np.where(
        (hurtos['ARMAS MEDIOS']=='CORTANTES') | 
        (hurtos['ARMAS MEDIOS']=='CORTOPUNZANTES') |
        (hurtos['ARMAS MEDIOS']=='PUNZANTES') |
        (hurtos['ARMAS MEDIOS']=='JERINGA'), 
        "NO REPORTADO", 
        hurtos['ARMAS MEDIOS'])
)

## BD Pobreza y Pobreza extrema

In [86]:
pobreza = pd.read_excel('./anexo_pobreza_monetaria_20_departamento.xls', sheet_name='IP Act.Met.', header=14)
pobreza = pobreza.iloc[:25,:].rename(columns={'Unnamed: 0':'DEPARTAMENTO'})
pobreza.columns = ['DEPARTAMENTO', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
pobreza = pobreza.melt(id_vars=['DEPARTAMENTO'], var_name='AÑO', value_name='pobreza')

In [87]:
pobreza_extrema= pd.read_excel('./anexo_pobreza_monetaria_20_departamento.xls', sheet_name='IPE Act.Met.', header=14)
pobreza_extrema=pobreza_extrema.iloc[:25,:].rename(columns={'Unnamed: 0':'DEPARTAMENTO'})
pobreza_extrema.columns = ['DEPARTAMENTO', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']
pobreza_extrema = pobreza_extrema.melt(id_vars=['DEPARTAMENTO'], var_name='AÑO', value_name='pobreza_extrema')

In [12]:
#pobreza_gini= pd.read_excel('./anexo_pobreza_monetaria_20_departamento.xls', sheet_name='Gini', header=14)
#pobreza_gini=pobreza_gini.iloc[:25,:].rename(columns={'Unnamed: 0':'Departamento'})
#pobreza_gini = pobreza_gini.melt(id_vars=['Departamento'], var_name='año', value_name='gini')

In [88]:
consolidado_pobreza = (
    pobreza.merge(
        pobreza_extrema,
        how='inner',
        on=['DEPARTAMENTO', 'AÑO']
    )
)

In [95]:
consolidado_pobreza['DEPARTAMENTO'] = consolidado_pobreza.DEPARTAMENTO.str.upper()

In [96]:
consolidado_pobreza = consolidado_pobreza.merge(divipola_dpto, how='inner', on='DEPARTAMENTO').drop(columns='DEPARTAMENTO')

## Desempleo

In [17]:
Desempleo = pd.read_excel('./anexo_dep_21.xls', sheet_name='Departamentos anual', header=12)
Desempleo = Desempleo.rename(columns={'Unnamed: 0':'indicador'})
Desempleo['indicador'] = Desempleo.indicador.str.upper()


In [18]:
Desempleo=Desempleo.merge(divipola_dpto, how='left', left_on='indicador', right_on='DEPARTAMENTO')

In [19]:
Desempleo['COD_DPTO']=Desempleo['COD_DPTO'].fillna(method='ffill')
Desempleo['DEPARTAMENTO']=Desempleo['DEPARTAMENTO'].fillna(method='ffill')

In [20]:
Desempleo = Desempleo[~Desempleo[2007].isnull()]
Desempleo = Desempleo[Desempleo[2007]!=2007]
Desempleo.rename(columns={'2020*':2020}, inplace=True)

In [21]:
Desempleo = Desempleo.melt(id_vars=['indicador','COD_DPTO', 'DEPARTAMENTO'], var_name='AÑO', value_name='indicador_f')

In [22]:
Desempleo = pd.pivot_table(Desempleo, values='indicador_f', index=['COD_DPTO', 'AÑO'], columns='indicador').reset_index()

In [23]:
Desempleo['AÑO'] = Desempleo.AÑO.astype(str)

## Poblacion

In [59]:
poblacion1 = pd.read_excel('./anexo-proyecciones-poblacion-Municipal_Area_2018-2035.xlsx', header=11)
poblacion2 = pd.read_excel('./anexo-area-proyecciones-poblacion-Municipal_Area_1985-2017.xlsx', header=11)

In [61]:
poblacion2 = poblacion2[~poblacion2['DPNOM'].isnull()]

In [62]:
poblacion2['AÑO']=round(poblacion2['AÑO'],0).astype(int)

In [63]:
poblacion1['COD_MUNIC']=poblacion1['DPMP'].astype(str).str.split(' ',expand=True)[0].astype(str).str.pad(5, fillchar='0')
poblacion2['COD_MUNIC']=round(poblacion2['COD_MUNICIPIO'],0).astype(int).astype(str).str.split(' ',expand=True)[0].astype(str).str.pad(5, fillchar='0')

In [64]:
poblacion1 = poblacion1[poblacion1['ÁREA GEOGRÁFICA']!='Total']
poblacion2 = poblacion2[poblacion2['ÁREA GEOGRÁFICA']!='Total']

poblacion1 = (
    poblacion1.groupby(['COD_MUNIC','AÑO']).agg(
        poblacion=pd.NamedAgg(column='Total', aggfunc="sum")).reset_index()
)

poblacion2 = (
    poblacion2.groupby(['COD_MUNIC','AÑO']).agg(
        poblacion=pd.NamedAgg(column='Total', aggfunc="sum")).reset_index()
)
poblacion=pd.concat([poblacion1,poblacion2], axis=0)
poblacion['AÑO'] = poblacion.AÑO.astype(str)

## Integración

In [97]:
hurtos=hurtos.merge(divipola, how='inner', on='COD_MUNIC')

In [98]:
hurtos=hurtos.merge(consolidado_pobreza, how='left', on=['COD_DPTO','AÑO'])

In [105]:
hurtos=hurtos.merge(poblacion, how='inner', on=['COD_MUNIC','AÑO'])
hurtos=hurtos.merge(Desempleo, how='left', on=['COD_DPTO','AÑO'])

In [106]:
hurtos['tasa_hurto_dptal']=hurtos['CANTIDAD']/hurtos['poblacion']

In [107]:
hurtos['tasahurtosx10000']=hurtos['tasa_hurto_dptal']*10000

In [108]:
var_final = ['ARMAS MEDIOS', 'FECHA HECHO', 'TIPO DE HURTO',
             'CANTIDAD', 'COD_MUNIC', 'AÑO', 'PERIODO', 'DIA', 
             'COD_DPTO','DEPARTAMENTO', 'MUNICIPIO', 'LATITUD', 'LONGITUD', 'pobreza',
             'pobreza_extrema', 'poblacion','TD', 'TGP', 'TO', 'tasa_hurto_dptal','tasahurtosx10000']

In [109]:
hurtos = hurtos[hurtos.AÑO>'2016']

# Expoirtación de los Datos

In [110]:
hurtos[var_final].to_csv('./hurtos_consolidado.csv', index=False)


In [73]:
events=hurtos.groupby(["FECHA HECHO"]).agg(
    CANTIDAD=pd.NamedAgg(column='CANTIDAD', aggfunc="sum"))#.reset_index()

In [111]:
gr_dptos=hurtos.groupby(["DEPARTAMENTO", 'AÑO']).agg(
    tasa_desempleo=pd.NamedAgg(column='TD', aggfunc="mean"),
    indice_pobreza=pd.NamedAgg(column='pobreza', aggfunc="mean"),
    indice_pobreza_extrema=pd.NamedAgg(column='pobreza_extrema', aggfunc="mean"),
    tasahurtosx10000=pd.NamedAgg(column='tasahurtosx10000', aggfunc="sum")
)

In [113]:
gr_dptos.reset_index(inplace=True)

In [116]:
gr_dptos.to_csv('./indices_socieconomicos_hurtos.csv', index=False)